In [4]:
import pandas as pd
import pandas_profiling as pp


In [5]:
import os
data_path = '/home/srgrace/genericContest_data/cat-in-the-dat'

train = pd.read_csv(os.path.join(data_path, 'train.csv'))
test = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [6]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [7]:
test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
1,300001,0,0,0,T,N,Red,Square,Lion,Canada,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
2,300002,1,0,1,F,Y,Blue,Square,Dog,China,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
3,300003,0,0,1,T,Y,Red,Star,Cat,China,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
4,300004,0,1,1,F,N,Red,Trapezoid,Dog,China,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [9]:
import warnings
warnings.filterwarnings("ignore")

import gc
import joblib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [10]:
test['target'] = -1
data = pd.concat([train, test]).reset_index(drop=True)

features = [x for x in train.columns if x not in ['id', 'target']]

for feat in features:
    label_enc = preprocessing.LabelEncoder()
    data[feat] = label_enc.fit_transform(data[feat].fillna('-1').astype(str).values)

In [13]:
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)

In [15]:
train.tail()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
299995,299995,0,0,0,1,0,2,4,5,4,...,10545,0,0,2,10,10,108,2,10,0
299996,299996,0,0,0,0,1,1,4,4,5,...,6025,1,4,2,7,22,40,2,4,0
299997,299997,0,0,0,0,1,0,3,0,5,...,2302,2,4,0,14,0,7,6,11,1
299998,299998,0,1,0,0,1,1,2,0,2,...,7983,0,3,0,7,22,174,2,10,1
299999,299999,0,0,0,0,1,0,4,2,5,...,8070,2,0,2,8,17,172,0,5,0


In [17]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [22]:
def create_model(data, feats):
    inps = []
    outs = []
    for feat in feats:
        num_unique_vals = int(data[feat].nunique())
        embed_dim = int(min(np.ceil(num_unique_vals/2), 50))
        
        inp = layers.Input(shape=(1, ))
        out = layers.Embedding(num_unique_vals + 1, embed_dim, name=feat)(inp)
        
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        
        inps.append(inp)
        outs.append(out)
    
    x = layers.Concatenate()(outs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation='softmax')(x)
    
    model = Model(inputs=inps, outputs=y)
    return model

In [24]:
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))


skf = StratifiedKFold(n_splits=50)
for train_idx, test_idx in skf.split(train, train.target.values):
    
    x_train, x_test = train.iloc[train_idx, :], train.iloc[test_idx, :]
    x_train = x_train.reset_index(drop=True)
    x_test = x_test.reset_index(drop=True)
    
    y_train, y_test = x_train.target.values, x_test.target.values
    
    model = create_model(data, features)
    model.compile(loss='binary_crossentropy', optimizers='adam', metrics=[auc])
    
    x_train = [x_train.loc[:, features].values[:, k] for k in range(x_train.loc[:, features].values.shape[1])]
    x_test = [x_test.loc[:, features].values[:, k] for k in range(x_test.loc[:, features].values.shape[1])]
    
    early_stop = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5, verbose=1, mode='max',
                                         baseline=None, restore_best_weights=True)
    
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, 
                                           mode='max', verbose=1)
    
    model.fit(x_train, utils.to_categorical(y_train),
             validation_data=(x_test, utils.to_categorical(y_test)),
             verbose=1, batch_size=1024, callbacks=[early_stop, reduce_lr], epochs=100)
    
    valid_fold_preds = model.predict(x_test)[:, 1]
    test_fold_preds = model.predict(test_data)[:, 1]
    oof_preds[test_idx] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    print(metrics.roc_auc_score(y_test, valid_fold_preds))
    K.clear_session()

Train on 293999 samples, validate on 6001 samples
Epoch 1/100
293999/293999 [==============================] - 17s 58us/sample - loss: 0.5783 - auc: 0.7050 - val_loss: 0.5708 - val_auc: 0.7890
Epoch 2/100
293999/293999 [==============================] - 11s 38us/sample - loss: 0.5026 - auc: 0.7824 - val_loss: 0.4922 - val_auc: 0.7971
Epoch 3/100
293999/293999 [==============================] - 12s 40us/sample - loss: 0.4858 - auc: 0.8009 - val_loss: 0.4939 - val_auc: 0.7967
Epoch 4/100
293999/293999 [==============================] - 11s 38us/sample - loss: 0.4758 - auc: 0.8110 - val_loss: 0.4950 - val_auc: 0.7932
Epoch 5/100
292864/293999 [============================>.] - ETA: 0s - loss: 0.4669 - auc: 0.8192
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
293999/293999 [==============================] - 11s 39us/sample - loss: 0.4669 - auc: 0.8191 - val_loss: 0.4958 - val_auc: 0.7914
Epoch 6/100
293999/293999 [==============================] - 10s 33us

293999/293999 [==============================] - 11s 37us/sample - loss: 0.4506 - auc: 0.8336 - val_loss: 0.5083 - val_auc: 0.7825
Epoch 00007: early stopping
0.7961919408286191
Train on 293999 samples, validate on 6001 samples
Epoch 1/100
293999/293999 [==============================] - 15s 50us/sample - loss: 0.5805 - auc: 0.7036 - val_loss: 0.5501 - val_auc: 0.7970
Epoch 2/100
293999/293999 [==============================] - 10s 33us/sample - loss: 0.5020 - auc: 0.7829 - val_loss: 0.5071 - val_auc: 0.8039
Epoch 3/100
293999/293999 [==============================] - 10s 34us/sample - loss: 0.4864 - auc: 0.8004 - val_loss: 0.4882 - val_auc: 0.8025
Epoch 4/100
293999/293999 [==============================] - 11s 36us/sample - loss: 0.4773 - auc: 0.8095 - val_loss: 0.4900 - val_auc: 0.8009
Epoch 5/100
292864/293999 [============================>.] - ETA: 0s - loss: 0.4692 - auc: 0.8172
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
293999/293999 [=======

293999/293999 [==============================] - 12s 40us/sample - loss: 0.4698 - auc: 0.8169 - val_loss: 0.5145 - val_auc: 0.7849
Epoch 6/100
293888/293999 [============================>.] - ETA: 0s - loss: 0.4619 - auc: 0.8238
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
293999/293999 [==============================] - 12s 40us/sample - loss: 0.4619 - auc: 0.8239 - val_loss: 0.5198 - val_auc: 0.7835
Epoch 7/100
293999/293999 [==============================] - 12s 40us/sample - loss: 0.4483 - auc: 0.8357 - val_loss: 0.5152 - val_auc: 0.7769
Epoch 00007: early stopping
0.7912693835704216
Train on 293999 samples, validate on 6001 samples
Epoch 1/100
293999/293999 [==============================] - 19s 64us/sample - loss: 0.5762 - auc: 0.7058 - val_loss: 0.5571 - val_auc: 0.7906
Epoch 2/100
293999/293999 [==============================] - 11s 39us/sample - loss: 0.5011 - auc: 0.7839 - val_loss: 0.5132 - val_auc: 0.7933
Epoch 3/100
293999/293999 [=======

294000/294000 [==============================] - 9s 32us/sample - loss: 0.5010 - auc: 0.7845 - val_loss: 0.4969 - val_auc: 0.8008
Epoch 3/100
294000/294000 [==============================] - 9s 32us/sample - loss: 0.4861 - auc: 0.8010 - val_loss: 0.5007 - val_auc: 0.7970
Epoch 4/100
294000/294000 [==============================] - 9s 32us/sample - loss: 0.4775 - auc: 0.8093 - val_loss: 0.5031 - val_auc: 0.7934
Epoch 5/100
293888/294000 [============================>.] - ETA: 0s - loss: 0.4688 - auc: 0.8175
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
294000/294000 [==============================] - 9s 32us/sample - loss: 0.4688 - auc: 0.8174 - val_loss: 0.4956 - val_auc: 0.7916
Epoch 6/100
294000/294000 [==============================] - 14s 47us/sample - loss: 0.4560 - auc: 0.8288 - val_loss: 0.4982 - val_auc: 0.7878
Epoch 7/100
294000/294000 [==============================] - 15s 51us/sample - loss: 0.4504 - auc: 0.8339 - val_loss: 0.5061 - val_auc:

Epoch 00007: early stopping
0.8164066062392261
Train on 294000 samples, validate on 6000 samples
Epoch 1/100
294000/294000 [==============================] - 16s 55us/sample - loss: 0.5856 - auc: 0.7022 - val_loss: 0.5521 - val_auc: 0.7911
Epoch 2/100
294000/294000 [==============================] - 9s 31us/sample - loss: 0.5024 - auc: 0.7827 - val_loss: 0.4939 - val_auc: 0.7965
Epoch 3/100
294000/294000 [==============================] - 9s 31us/sample - loss: 0.4865 - auc: 0.8000 - val_loss: 0.4989 - val_auc: 0.7941
Epoch 4/100
294000/294000 [==============================] - 11s 38us/sample - loss: 0.4775 - auc: 0.8094 - val_loss: 0.4990 - val_auc: 0.7951
Epoch 5/100
293888/294000 [============================>.] - ETA: 0s - loss: 0.4688 - auc: 0.8176
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
294000/294000 [==============================] - 11s 38us/sample - loss: 0.4688 - auc: 0.8173 - val_loss: 0.4959 - val_auc: 0.7923
Epoch 6/100
294000/29400

294000/294000 [==============================] - 12s 40us/sample - loss: 0.4515 - auc: 0.8328 - val_loss: 0.4946 - val_auc: 0.7930
Epoch 00007: early stopping
0.8103205969036116
Train on 294000 samples, validate on 6000 samples
Epoch 1/100
294000/294000 [==============================] - 17s 58us/sample - loss: 0.5810 - auc: 0.7046 - val_loss: 0.5602 - val_auc: 0.7896
Epoch 2/100
294000/294000 [==============================] - 10s 35us/sample - loss: 0.5012 - auc: 0.7842 - val_loss: 0.4985 - val_auc: 0.7922
Epoch 3/100
294000/294000 [==============================] - 11s 37us/sample - loss: 0.4864 - auc: 0.8006 - val_loss: 0.5015 - val_auc: 0.7919
Epoch 4/100
294000/294000 [==============================] - 12s 42us/sample - loss: 0.4770 - auc: 0.8101 - val_loss: 0.5008 - val_auc: 0.7907
Epoch 5/100
293888/294000 [============================>.] - ETA: 0s - loss: 0.4692 - auc: 0.8171
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
294000/294000 [=======

293888/294000 [============================>.] - ETA: 0s - loss: 0.4698 - auc: 0.8168
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
294000/294000 [==============================] - 11s 39us/sample - loss: 0.4698 - auc: 0.8167 - val_loss: 0.5048 - val_auc: 0.7804
Epoch 6/100
294000/294000 [==============================] - 10s 34us/sample - loss: 0.4555 - auc: 0.8291 - val_loss: 0.5116 - val_auc: 0.7716
Epoch 7/100
294000/294000 [==============================] - 12s 39us/sample - loss: 0.4507 - auc: 0.8340 - val_loss: 0.5201 - val_auc: 0.7707
Epoch 00007: early stopping
0.7913701502399324
Train on 294000 samples, validate on 6000 samples
Epoch 1/100
294000/294000 [==============================] - 25s 84us/sample - loss: 0.5809 - auc: 0.7058 - val_loss: 0.5684 - val_auc: 0.7878
Epoch 2/100
294000/294000 [==============================] - 14s 49us/sample - loss: 0.5010 - auc: 0.7844 - val_loss: 0.4998 - val_auc: 0.7972
Epoch 3/100
294000/294000 [=======

294001/294001 [==============================] - 11s 39us/sample - loss: 0.5020 - auc: 0.7830 - val_loss: 0.4921 - val_auc: 0.8127
Epoch 3/100
294001/294001 [==============================] - 11s 36us/sample - loss: 0.4869 - auc: 0.7998 - val_loss: 0.4796 - val_auc: 0.8138
Epoch 4/100
294001/294001 [==============================] - 10s 36us/sample - loss: 0.4771 - auc: 0.8095 - val_loss: 0.4832 - val_auc: 0.8073
Epoch 5/100
294001/294001 [==============================] - 10s 36us/sample - loss: 0.4693 - auc: 0.8174 - val_loss: 0.4897 - val_auc: 0.8052
Epoch 6/100
292864/294001 [============================>.] - ETA: 0s - loss: 0.4621 - auc: 0.8234
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
294001/294001 [==============================] - 10s 36us/sample - loss: 0.4622 - auc: 0.8231 - val_loss: 0.4909 - val_auc: 0.7990
Epoch 7/100
294001/294001 [==============================] - 10s 35us/sample - loss: 0.4488 - auc: 0.8353 - val_loss: 0.5032 - val_

294001/294001 [==============================] - 10s 34us/sample - loss: 0.4494 - auc: 0.8346 - val_loss: 0.5072 - val_auc: 0.7824
Epoch 00007: early stopping
0.7955495004541326
Train on 294001 samples, validate on 5999 samples
Epoch 1/100
294001/294001 [==============================] - 15s 52us/sample - loss: 0.5810 - auc: 0.7046 - val_loss: 0.5644 - val_auc: 0.7883
Epoch 2/100
294001/294001 [==============================] - 10s 35us/sample - loss: 0.5015 - auc: 0.7837 - val_loss: 0.4903 - val_auc: 0.7950
Epoch 3/100
294001/294001 [==============================] - 10s 35us/sample - loss: 0.4861 - auc: 0.8007 - val_loss: 0.5029 - val_auc: 0.7959
Epoch 4/100
294001/294001 [==============================] - 10s 35us/sample - loss: 0.4766 - auc: 0.8099 - val_loss: 0.4946 - val_auc: 0.7918
Epoch 5/100
294001/294001 [==============================] - 10s 35us/sample - loss: 0.4694 - auc: 0.8167 - val_loss: 0.5020 - val_auc: 0.7907
Epoch 6/100
293888/294001 [============================>.

In [25]:
print("Overall AUC={}".format(metrics.roc_auc_score(train.target.values, oof_preds)))

Overall AUC=0.7945786383599281


In [26]:
test_preds /= 50
test_ids = test.id.values
print("Saving submission file")
submission = pd.DataFrame.from_dict({
    'id': test_ids,
    'target': test_preds
})
submission.to_csv("data/submission-1.csv", index=False)

Saving submission file
